# The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
import glob

# The workers

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(threads_per_worker=1)
c = Client(cluster)
c

/scratch/cnt0024/hmg2840/albert7a/DEV/git/conda-pack/perf-pangeo/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40466 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:33289 Dashboard: http://127.0.0.1:40466/status,Cluster Workers: 56 Cores: 56 Memory: 270.19 GB


# The data

In [3]:
files=sorted(glob.glob('/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02*-S/*/eNATL60*gridT-2D*.nc'))
drop_vars = ['nav_lat', 'nav_lon', 'somxl010', 'sosaline', 'sosstsst']
extra_coord_vars = []
chunks = dict(time_counter=240, y=240, x=480)
open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),
                   chunks=chunks,decode_cf=True,decode_times=False,
                   concat_dim="time_counter",combine='nested')


In [4]:
%time ds = xr.open_mfdataset(files, parallel=True, **open_kwargs)

CPU times: user 4.12 s, sys: 997 ms, total: 5.11 s
Wall time: 7.99 s


In [5]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 1.28 s, sys: 32.7 ms, total: 1.31 s
Wall time: 1.29 s


In [ ]:
%time mean.load()

In [ ]:
cluster.close()